In [1]:
from Helper.ml_models import *
from Helper.Helper_functions import * 
from torchvision.models.segmentation import *
import gc, sys
from torch.utils.tensorboard import SummaryWriter

In [2]:
model = 'deeplabv3_resnet50'
writer = SummaryWriter(f'K_Fold_Run/{model}_k_fold_{0}/logs')

trained_model = TrainedModel(model,
                                        2048,
                                        1024,
                                        folder_path='K_Fold_Run',
                                        weights_name=f'{model}_k_fold_{0}',
                                        start_epoch='latest', 
                                        writer=writer, 
                                        )
k_fold_dataset = K_Fold_Dataset(image_dir='CityscapesDaten/images',
                                        annotation_dir='CityscapesDaten/semantic',
                                        k_fold_csv_dir='Daten/CityscapesDaten',
                                        leave_out_fold=0, 
                                        )    
k_fold_dataset.check_for_data_leaks()               
        
trained_model.prepare_model_training(dataset_train=k_fold_dataset.train_dataset,
                                    dataset_val=k_fold_dataset.val_dataset,
                                    dataset_test=k_fold_dataset.test_dataset,
                                    batch_size=2, 
                                    shuffle=True, 
                                    learning_rate=1*10**(-7), 
                                    momentum=0.9,
                                    weight_decay=0.00001, 
                                    num_workers=4, 
                                    pin_memory=True,
                                    )

Using CUDA GPU
Model loaded: deeplabv3_resnet50 | Device: cuda 
No data leaks found.
Training Dataset prepared
Validation Dataset prepared
Test Dataset prepared


In [3]:
for i in range(5):
    trained_model.inference_tensorboard(0)
    trained_model.epoch +=1

In [4]:
trained_model.test()

Test Loss: 2.048
Test Accuracy: 63.07 %


In [5]:
trained_model.auto_train(epochs=1)

Epoch 1 von 1


  3%|▎         | 34/1112 [00:05<02:56,  6.09it/s]


KeyboardInterrupt: 